In [1]:
code="""
#from string import *
def f(x):
    print 'from inside',globals().keys()
    return x**2
"""


In [20]:
#%%writefile insulating_exec.py
'''
make_function takes a function source string as input and returns a pointer to the executable function

Parameters: 
        name: name of the function (should be consistent with the name used in the code)
        code: A string containing the code

Returns:
        if type==function: a pointer to the executable function
        if type==str: An error message
'''
import sys
import StringIO

def make_function(name,code):
    #locals().pop(name,'clearing keypair')
    try:
        bytecode=compile(source=code,filename='filter source code',mode='exec')
        env={}

        stdout = sys.stdout  #keep a handle on the real standard output
        #sys.stdout = StringIO.StringIO() #Choose a file-like object to write to
        print 'test1'
        exec(bytecode,env)
        print 'test2'
        
        print 'env keys=',env.keys()

    except SyntaxError, e:
        message='Syntax error:  line=%d, offset=%d text=%s'%(e.lineno,e.offset,e.text)
        return message
    
    if env.has_key(name):
        return env[name],env
    else:
        return 'code failed to generate function named '+name

    
if __name__=="__main__":  
    code="""
#from string import *
def f(x):
    print 'from inside',globals().keys()
    return x**2
"""
    A,env=make_function('f',code)
    if type(A)==str:
        print A
    else:
        old_stdout=sys.stdout
        sys.stdout=StringIO.StringIO()
        env['A']=A
        exec('x=A(2)',env)
        stdout_dump=sys.stdout.getvalue()
        sys.stdout=old_stdout
        print A, type(A)
        print 'stdout_dump=|%s|'%stdout_dump
        print 'returned value=',env['x']
    

    

test1
test2
env keys= ['__builtins__', 'f']
<function f at 0x105e21c80> <type 'function'>
stdout_dump=|from inside ['__builtins__', 'A', 'f']
|
returned value= 4


In [18]:
import sys
import StringIO

print 'before redirect'
old_stdout=sys.stdout
sys.stdout=StringIO.StringIO()
exec("print 'after redirect'",{})
text=sys.stdout.getvalue()
sys.stdout=old_stdout
print 'text=',text

before redirect
text= after redirect



In [7]:
from sys import stdout
globals()['stdout']
old_stdout=stdout
stdout=StringIO.StringIO()

In [10]:
stdout.getvalue()

''

In [37]:
# %load tmp
env keys= ['ascii_lowercase', 'upper', 'punctuation', 'letters', 'lstrip', 'uppercase', 'ascii_letters', 'replace', 'capwords', 'index_error', 'rfind', 'Template', 'strip', 'ljust', 'octdigits', 'index', 'ascii_uppercase', 'whitespace', 'rindex', 'rsplit', 'atol_error', 'find', 'printable', 'splitfields', 'split', 'rstrip', 'translate', 'digits', 'joinfields', 'rjust', '__builtins__', 'swapcase', 'atof_error', 'atoi', 'hexdigits', 'atol', 'atof', 'lowercase', 'count', 'lower', 'join', 'center', 'f', 'expandtabs', 'atoi_error', 'maketrans', 'capitalize', 'zfill', 'Formatter']
<function f at 0x106e03668> <type 'function'>
2
4


SyntaxError: invalid syntax (<ipython-input-37-4204c6a82c29>, line 2)

In [4]:
globals()['stdout']

In [7]:
[A[i](i) for i in range(len(A))]

0
1 1
2
3 3


[0, -1, 4, -3]

### communicating through pipes

In [1]:
from multiprocessing import Process, Pipe

def f(conn1,conn2):
    #sys.stdout=conn2
    while True:
        string=conn1.recv()
        conn1.send(string+string)
        conn2.send('f recieved '+string)
        # print 'f recieved '+string
        # sys.stdout.flush()

if __name__ == '__main__':
    parent_conn1, child_conn1 = Pipe()
    parent_conn2, child_conn2 = Pipe()
    p = Process(target=f, args=(child_conn1,child_conn2))
    p.start()
    for i in range(10):
        parent_conn1.send(str(i))
        print parent_conn1.recv()
        if parent_conn2.poll():
            print "master got through conn2",parent_conn2.recv()

    parent_conn1.close()
    parent_conn2.close()

00
master got through conn2 f recieved 0
11
master got through conn2 f recieved 1
22
master got through conn2 f recieved 2
33
master got through conn2 f recieved 3
44
master got through conn2 f recieved 4
55
master got through conn2 f recieved 5
66
master got through conn2 f recieved 6
77
master got through conn2 f recieved 7
88
master got through conn2 f recieved 8
99
master got through conn2 f recieved 9


In [2]:
parent_conn1.recv?

In [70]:
from contextlib import contextmanager

@contextmanager
def stdout_redirector(stream):
    old_stdout = sys.stdout
    sys.stdout = stream
    try:
        yield
    finally:
        sys.stdout = old_stdout


In [71]:
import io
f = io.StringIO()
with stdout_redirector(f):
    #print 'foobar'
    print 12
print(u'Got stdout: "{0}"'.format(f.getvalue()))

TypeError: unicode argument expected, got 'str'

In [29]:
%%writefile ../TimeoutError.py
from functools import wraps
import errno
import os
import signal

class TimeoutError(Exception):
    pass

def timeout(seconds=1, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

Writing ../TimeoutError.py


In [30]:
from time import sleep
@timeout(1)
def long_running_function():
    for i in range(20):
        print i,
        sleep(0.1)

try:        
    long_running_function()
except Exception, e:
    print e


0 1 2 3 4 5 6 7 8 9 STREAM ioctl timeout


In [6]:
from os import wait

In [25]:
import StringIO

output = StringIO.StringIO()
output.write('First line.\n')
print >>output, 'Second line.'

# Retrieve file contents -- this will be
# 'First line.\nSecond line.\n'
contents = output.getvalue()

# Close object and discard memory buffer --
# .getvalue() will now raise an exception.
output.close()

In [27]:
print contents

In [12]:
locals()

{'A': <function f>,
 'In': ['',
  u'code="""\nfrom string import *\ndef f(x):\n    print x\n    return x**2\n"""',
  u"'''\nmake_function takes a function source string as input and returns a pointer to the executable function\n\nParameters: \n        name: name of the function (should be consistent with the name used in the code)\n        code: A string containing the code\n\nReturns:\n        if type==function: a pointer to the executable function\n        if type==str: An error message\n'''\nimport sys\nimport StringIO\n\ndef make_function(name,code):\n    #locals().pop(name,'clearing keypair')\n    try:\n        bytecode=compile(source=code,filename='filter source code',mode='exec')\n        env={}\n        #dump=StringIO.StringIO()\n        #old_stdout=sys.stdout\n        #sys.stdout=dump\n        exec(bytecode,env)\n        #sys.stdout=old_stdout\n        \n        #print 'dump=',dump.getvalue()\n        print 'env keys=',env.keys()\n\n    except SyntaxError, e:\n        message=

In [31]:
import logging

# create logger
logger = logging.getLogger('simple_example')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warn('warn message')
logger.error('error message')
logger.critical('critical message')

2015-09-19 13:12:29,682 - simple_example - DEBUG - debug message
DEBUG:simple_example:debug message
2015-09-19 13:12:29,683 - simple_example - INFO - info message
INFO:simple_example:info message
2015-09-19 13:12:29,684 - simple_example - WARNING - warn message
2015-09-19 13:12:29,685 - simple_example - ERROR - error message
ERROR:simple_example:error message
2015-09-19 13:12:29,686 - simple_example - CRITICAL - critical message
CRITICAL:simple_example:critical message


In [33]:
env.keys()

['__builtins__', 'A', 'x', 'f']

In [35]:
env.pop('A')

KeyError: 'A'